In [664]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
from kneed import DataGenerator, KneeLocator
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [665]:
df1 = pd.read_feather("Total_BG.feather")

In [666]:
df1 = df1[df1["Date"]>="20200101"]
df1 = df1[df1["Date"]<="20220101"]
df1.set_index("Date", drop=True, inplace=True)
df1.index = pd.to_datetime(df1.index)

                           Solar  Wind Onshore  Renewable Total   Load_  \
Date                                                                      
2020-01-01 01:00:00+02:00    0.0           0.0              0.0  4243.0   
2020-01-01 02:00:00+02:00    0.0         323.0            323.0  4063.0   
2020-01-01 03:00:00+02:00    0.0         306.0            306.0  3924.0   
2020-01-01 04:00:00+02:00    0.0         351.0            351.0  3838.0   
2020-01-01 05:00:00+02:00    0.0         431.0            431.0  3795.0   
...                          ...           ...              ...     ...   
2021-12-31 20:00:00+02:00    0.0         137.0            137.0  4790.0   
2021-12-31 21:00:00+02:00    0.0         145.0            145.0  4343.0   
2021-12-31 22:00:00+02:00    0.0         150.0            150.0  4196.0   
2021-12-31 23:00:00+02:00    0.0         136.0            136.0  4147.0   
2022-01-01 00:00:00+02:00    0.0         219.0            219.0  4181.0   

                        

In [667]:
Before_MC = df1[df1.index <="20210511"]

In [668]:
After_MC = df1[df1.index>="20210511"]

In [669]:
Before_MC_X_train=Before_MC[Before_MC.index<="20200511"].drop(["Prices_BG"],axis=1)
Before_MC_y_train=Before_MC[Before_MC.index<="20200511"]["Prices_BG"]

Before_MC_X_test=Before_MC[Before_MC.index>="20200511"].drop(["Prices_BG"],axis=1)
Before_MC_y_test=Before_MC[Before_MC.index>="20200511"]["Prices_BG"]

After_MC_X_train=After_MC[After_MC.index<="20210511"].drop(["Prices_BG"],axis=1)
After_MC_y_train=After_MC[After_MC.index<="20210511"]["Prices_BG"]

After_MC_X_test=After_MC[After_MC.index>="20210511"].drop(["Prices_BG"],axis=1)
After_MC_y_test=After_MC[After_MC.index>="20210511"]["Prices_BG"]

In [670]:
# Before_MC_X_train.index = pd.to_numeric(Before_MC_X_train.index)
# Before_MC_X_test.index = pd.to_numeric(Before_MC_X_test.index)

# After_MC_X_train.index = pd.to_numeric(After_MC_X_train.index)
# After_MC_X_test.index = pd.to_numeric(After_MC_X_test.index)


In [671]:
modelLR = LinearRegression()

# print(Before_MC_X_train)
# print(Before_MC_y_train)

modelLR.fit(Before_MC_X_train,Before_MC_y_train)
prediction = modelLR.predict(Before_MC_X_test)
print(mean_squared_error(Before_MC_y_test, prediction))
print(mean_absolute_error(Before_MC_y_test, prediction))
print(np.sqrt(mean_squared_error(Before_MC_y_test, prediction)))

prediction = pd.Series(prediction, index=Before_MC_y_test.index)
Before_MC_y_test = pd.concat([Before_MC_y_test, prediction], axis=1)
Before_MC_y_test.plot()

49733.15303432642
42.93279327070487
223.00931154175248


In [672]:
modelLR.score(Before_MC_X_test,Before_MC_y_test)
# modelLR.score(Before_MC_X_train,Before_MC_y_train)


-0.17789913377770983

In [673]:
modelLR1 = LinearRegression()

modelLR1.fit(After_MC_X_train,After_MC_y_train)

LinearRegression()

In [674]:
modelLR1.score(After_MC_X_test,After_MC_y_test)

-0.7495838857653683

In [675]:
After_MC_X_train

,Solar,Wind Onshore,Renewable Total,Load_,Forecasted Load,Net_Position_BG,Solar_FC,Wind Onshore_FC
Date,,,,,,,,
2021-05-11 00:00:00+02:00,0.0,30.0,30.0,3126.0,3079.0,0.0,0.0,7.0


In [676]:
After_MC_X_test

,Solar,Wind Onshore,Renewable Total,Load_,Forecasted Load,Net_Position_BG,Solar_FC,Wind Onshore_FC
Date,,,,,,,,
2021-05-11 00:00:00+02:00,0.0,30.0,30.0,3126.0,3079.0,0.0,0.0,7.0
2021-05-11 01:00:00+02:00,0.0,30.0,30.0,2992.0,2978.0,0.0,0.0,4.0
2021-05-11 02:00:00+02:00,0.0,26.0,26.0,2956.0,2937.0,0.0,0.0,2.0
2021-05-11 03:00:00+02:00,0.0,23.0,23.0,2987.0,2942.0,0.0,0.0,1.0
2021-05-11 04:00:00+02:00,0.0,25.0,25.0,3140.0,3024.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2021-12-31 20:00:00+02:00,0.0,137.0,137.0,4790.0,4839.0,744.0,0.0,204.0
2021-12-31 21:00:00+02:00,0.0,145.0,145.0,4343.0,4602.0,756.0,0.0,211.0
2021-12-31 22:00:00+02:00,0.0,150.0,150.0,4196.0,4558.0,700.0,0.0,204.0


In [677]:
After_MC_y_test

Date
2021-05-11 00:00:00+02:00    135.62
2021-05-11 01:00:00+02:00    129.32
2021-05-11 02:00:00+02:00    125.46
2021-05-11 03:00:00+02:00    121.70
2021-05-11 04:00:00+02:00    122.31
                              ...  
2021-12-31 20:00:00+02:00    221.01
2021-12-31 21:00:00+02:00    205.37
2021-12-31 22:00:00+02:00    194.35
2021-12-31 23:00:00+02:00    205.38
2022-01-01 00:00:00+02:00    113.39
Name: Prices_BG, Length: 5641, dtype: float64

In [678]:
modelLR1.predict(After_MC_X_test)

array([135.62, 135.62, 135.62, ..., 135.62, 135.62, 135.62])

In [679]:
coef_df = pd.DataFrame()
coef_df["Name"] = Before_MC_X_train.columns
coef_df["Coef"] = modelLR.coef_
coef_df.sort_values(by="Coef",ascending=False,inplace=True)

In [680]:
coef_sum = coef_df["Coef"].sum()

In [681]:
coef_df["Coef"] = coef_df["Coef"].apply(lambda x: x/coef_sum)

In [682]:
np.array(coef_df["Coef"]).reshape(8,1)

array([[-0.46325434],
       [-0.1195403 ],
       [-0.05819628],
       [ 0.14611491],
       [ 0.17719292],
       [ 0.23538919],
       [ 0.41177011],
       [ 0.67052379]])

In [683]:
scaler = StandardScaler()
scaler.fit(np.array(coef_df["Coef"]).reshape(8,1))

StandardScaler()

In [684]:
coef_df["Coef"]=scaler.transform(np.array(coef_df["Coef"]).reshape(8,1))

In [685]:
coef_df

,Name,Coef
3,Load_,-1.821263
4,Forecasted Load,-0.757108
0,Solar,-0.567184
5,Net_Position_BG,0.065373
2,Renewable Total,0.161592
1,Wind Onshore,0.341770
6,Solar_FC,0.887854
7,Wind Onshore_FC,1.688968


In [686]:
modelCat = CatBoostRegressor()

modelCat.fit(Before_MC_X_train,Before_MC_y_train)
modelCat.score(Before_MC_X_test,Before_MC_y_test)

NameError: name 'CatBoostReBGessor' is not defined

In [ ]:
modelDTR = DecisionTreeRegressor()
modelDTR.fit(Before_MC_X_train,Before_MC_y_train)
modelDTR.score(Before_MC_X_test,Before_MC_y_test)

-2.6297774103403238